In [3]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [61]:
import pandas as pd

import google.datalab.bigquery as bq
from google.cloud import bigquery

In [5]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [40]:
BUCKET='going-tfx'

# Airline Ontime Data

## Flights from Atlanta in the months of June
Here, we explore the biqquery dataset ```bigquery-samples.airline_ontime_data```. To have a good start with a representativ subset of the ~70M flights recorded there, we first look at Atlanta during the months of June in any year.

### Airlines departing from Atlanta
We count 20 distinct airlines departing from Atlanta

In [8]:
%%bigquery 
select
  count(*) as num, c.code, f.airline as tlc, c.airline as name
from
  `bigquery-samples.airline_ontime_data.flights` f left outer join
  `bigquery-samples.airline_ontime_data.airline_id_codes` c
  on f.airline_code = c.code 
where
  extract(month from parse_date( "%Y-%m-%d", f.date)) = 6
  and 
  f.departure_airport='ATL'
group by c.code, f.airline, c.airline 
order by num desc

,num,code,tlc,name
0,165691,19790,DL,Delta Air Lines Inc.: DL
1,96164,20366,EV,ExpressJet Airlines Inc.: EV
2,61429,20437,FL,AirTran Airways Corporation: FL
3,6703,19805,AA,American Airlines Inc.: AA
4,5803,20417,OH,Comair Inc.: OH
5,3411,20355,US,US Airways Inc.: US (Merged with America West ...
6,3203,19704,CO,Continental Air Lines Inc.: CO
7,2948,20363,9E,Pinnacle Airlines Inc.: 9E
8,2883,19386,NW,Northwest Airlines Inc.: NW
9,2310,19977,UA,United Air Lines Inc.: UA


These are the 344 destination airports for all flights from atlanta (showing only the first 10):

In [23]:
query_all_destinations="""
select 
  count(*) as num_flights, f.arrival_airport, f.arrival_lat, f.arrival_lon 
from 
  `bigquery-samples.airline_ontime_data.flights` f
group by f.arrival_airport , f.arrival_lat, f.arrival_lon 
order by num_flights desc
"""
all_destinations = bq.Query(query_all_destinations).execute().result().to_dataframe()
print ("Number of destinations: {}".format(len(all_destinations)))
all_destinations[:10]


Number of destinations: 344


,num_flights,arrival_airport,arrival_lat,arrival_lon
0,4167691,ATL,33.63,-84.42
1,3601331,ORD,41.98,-87.90
2,3110538,DFW,32.89,-97.03
3,2301335,LAX,33.94,-118.40
4,2211125,DEN,39.86,-104.67
5,2001204,IAH,29.98,-95.34
6,1996060,PHX,33.43,-112.01
7,1701440,LAS,36.08,-115.15
8,1485915,DTW,42.21,-83.35
9,1442367,SFO,37.61,-122.37


---
### Wheather stations for all airports

Let's find the closest weather station each of the US aiports above. This is pretty involved but should be clear after close examination.

In [47]:
query_closest_stations="""
with all_pairs as (
  with 
  -------------------------------------------------------------------------
  --  unique airports
  -------------------------------------------------------------------------    
  airports as 
      (select 
        f.arrival_airport as tlc, f.arrival_lat, f.arrival_lon 
      from 
        `bigquery-samples.airline_ontime_data.flights` f
      group by f.arrival_airport , f.arrival_lat, f.arrival_lon),
    
  -------------------------------------------------------------------------
  --  weather stations
  -------------------------------------------------------------------------    
  stations as
      (SELECT 
        name as name, wban, lat, lon 
       FROM 
        `bigquery-public-data.noaa_gsod.stations`
      where
        lat is not null and
        lon is not null and
        wban != '99999' and
        country = 'US')      

  -------------------------------------------------------------------------
  --  Pairing any airport with any station while calculating the distance
  -------------------------------------------------------------------------    
  select
    a.tlc as airport, 
    array_agg (struct(
        w.name as station, 
        w.wban as wban,
        --- euclidean distance is perfectly fine for this case 
        sqrt(pow(a.arrival_lat - w.lat,2) + pow(a.arrival_lon - w.lon,2)) as distance
    )) as st_dist
  from
    airports a,
    stations w
  group by a.tlc
)

-------------------------------------------------------------------------
--  sort the station groups by their respective distance in ascending 
--  order and take only the first record
-------------------------------------------------------------------------    
select 
    airport, 
    (select as struct wban, distance from unnest(st_dist) order by distance asc limit 1).wban,

    -- you would omit these lines in production. I let it in for illustration
    (select as struct station, distance from unnest(st_dist) order by distance asc limit 1).station,
    (select as struct station, distance from unnest(st_dist) order by distance asc limit 1).distance
from all_pairs 
"""
nearest_stations = bq.Query(query_closest_stations).execute().result().to_dataframe()
print ("Number of pairs: {}".format(len(closest_stations)))
nearest_stations[:20]

Number of pairs: 344


,airport,wban,station,distance
0,SWF,14714,STEWART INTERNATIONAL AIRPORT,0.000
1,TEX,03011,TELLURIDE REGIONAL AIRPORT,0.000
2,GTR,53893,GOLDEN TRIANGLE RGNL APT,0.007
3,BQK,93836,BRUNSWICK GLYNN CO A,0.007
4,SUN,94161,FRIEDMAN MEMORIAL AIRPORT,0.010
5,OME,26617,NOME AIRPORT,0.001
6,CIC,93203,CHICO MUNICIPAL AIRPORT,0.010
7,IAH,12960,G BUSH INTERCONTINENTAL AP/HO,0.020
8,SGF,13995,SPRINGFIELD-BRANSON REGIONAL,0.010
9,ATL,13874,HARTSFIELD-JACKSON ATLANTA IN,0.022


Hard work! For a sanity check, you can plausibly tell that most station names match their airport name to some degree. 

Now let's create a table in Bigquery to hold this data:

---
### Create a link table in Bigquery with the bq tool

In [48]:
TMP_FILE="/tmp/airports_and_stations.csv"
TMP_FILE_GS="gs://" + BUCKET + TMP_FILE
TMP_FILE_GS

'gs://going-tfx/tmp/airports_and_stations.csv'

In [49]:
nearest_stations.to_csv(TMP_FILE, index=None)

In [50]:
!head -2 $TMP_FILE

airport,wban,station,distance
SWF,14714,STEWART INTERNATIONAL AIRPORT,0.0


In [45]:
!gsutil cp $TMP_FILE $TMP_FILE_GS

Copying file:///tmp/airports_and_stations.csv [Content-Type=text/csv]...
/ [1 files][ 18.6 KiB/ 18.6 KiB]                                                
Operation completed over 1 objects/18.6 KiB.                                     


In [46]:
!bq --location=US load --autodetect --source_format=CSV examples.AIRPORTS_STATIONS $TMP_FILE_GS

Waiting on bqjob_r64ea06f75d13ecba_00000166d987183c_1 ... (0s) Current status: DONE   


---
### Collecting the training data: ATL_JUNE

With the link table in place we can now get the weather info at origin and destination for all flights. 
The below query joins over weather, flights and the link table. It'll retrieve about 300'000 rows. To avoid crashing jupyterlab with that much data, we write it to a file and pipe it into the bq cli. An alternative would be to write a Beam pipeline with Bigquery source and sink and execute it with the DataflowRunner.

In [110]:
%%writefile /tmp/atl_june_query.sql
select 
  f.date, w_dep.year, w_dep.month, w_dep.day,
  extract(DAYOFWEEK from parse_date( "%Y-%m-%d", f.date)) as DEP_DOW,

  c.airline as AIRLINE,

  f.departure_schedule as DEP_T,
  f.departure_airport as DEP,
  f.departure_lat as DEP_LAT,
  f.departure_lon as DEP_LON,
  axs_dep.station DEP_W,  
  w_dep.mean_temp as MEAN_TEMP_DEP,
  w_dep.mean_visibility as MEAN_VIS_DEP,
  w_dep.mean_wind_speed as WND_SPD_DEP,

  f.arrival_schedule as ARR_T,
  f.arrival_delay as ARR_DELAY,
  f.arrival_airport as ARR, 
  f.arrival_lat as ARR_LAT,
  f.arrival_lon as ARR_LON,
  axs_arr.station ARR_W,  
  w_arr.mean_temp as MEAN_TEMP_ARR,
  w_arr.mean_visibility as MEAN_VIS_ARR,
  w_arr.mean_wind_speed as WND_SPD_ARR
  
from
  `bigquery-samples.airline_ontime_data.flights` f 
  join `going-tfx.examples.AIRPORTS_STATIONS` 
    axs_dep on f.departure_airport = axs_dep.airport
  join `going-tfx.examples.AIRPORTS_STATIONS` axs_arr 
    on f.arrival_airport = axs_arr.airport
  join `bigquery-samples.weather_geo.gsod` w_dep 
    on axs_dep.wban = w_dep.wban_number 
  join `bigquery-samples.weather_geo.gsod` w_arr
    on axs_arr.wban = w_arr.wban_number 
  join `bigquery-samples.airline_ontime_data.airline_id_codes` c
    on f.airline_code = c.code 

where
  extract(year from parse_date( "%Y-%m-%d", f.date)) = w_dep.year and
  extract(month from parse_date( "%Y-%m-%d", f.date)) = w_dep.month and
  extract(day from parse_date( "%Y-%m-%d", f.date)) = w_dep.day and

  extract(year from parse_date( "%Y-%m-%d", f.date)) = w_arr.year and
  extract(month from parse_date( "%Y-%m-%d", f.date)) = w_arr.month and
  extract(day from parse_date( "%Y-%m-%d", f.date)) = w_arr.day and

  extract(month from parse_date( "%Y-%m-%d", f.date)) = 6 and
  f.departure_airport = 'ATL'
order by
  f.date,
  f.departure_schedule 

Writing /tmp/atl_june_query.sql


We direct the output into a dummy variable not to pollute this notebook with the massive stdout pouring out of bq on this task. The job can take a couple of seconds. So be patient.

In [111]:
_ = !cat /tmp/atl_june_query.sql | bq --location=US query --destination_table examples.ATL_JUNE_RAW --replace --use_legacy_sql=false